In [45]:
import pandas as pd
import re
from pymystem3 import Mystem
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
import nltk
import numpy as np

from sklearn.model_selection import cross_val_score, KFold, ShuffleSplit, train_test_split
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import f1_score
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from numpy import argmax
from matplotlib import pyplot as plt

In [3]:
# Чтобы ошибки не мозолили глаза
import warnings
warnings.filterwarnings('ignore')

# Задание

Постройте классификатор, предсказывающий `category_id` на основании текстовой переменной `item_name` и категориальных переменных `item_price` и/или `brands`. 
Значения переменной item_name необходимо предобработать, рассмотреть необходимость использования лемматизации и н-грамм, построить векторное представление [TF-IDF](https://ru.wikipedia.org/wiki/TF-IDF) и подобрать оптимальный размер словаря. 
Категориальные переменные при необходимости так же нужно закодировать. 
Для обработанных признаков необходимо выбрать модель/модели и настроить их параметры. Дополнительно можно рассмотреть построение ансамблей алгоритмов.

In [4]:
data = pd.read_csv('data.csv')
data.head()

,receipt_id,receipt_dayofweek,receipt_time,item_name,item_quantity,item_price,item_nds_rate,category_id,brands
0,11,6,20:34,"Молоко 3,2%,шт",2.0,8,2,78,NaN
1,39,4,11:28,"Компот из изюма, 114 ккал",1.0,4,1,71,NaN
2,39,4,11:28,"Макаронные изделия отварные (масло сливочное),...",1.0,4,1,71,NaN
3,56,5,11:42,Кофе Капучино Большой Эден 18,1.0,12,1,70,NaN
4,105,3,01:53,Хлеб на СЫВОРОТКЕ 350г,1.0,7,-1,84,NaN


In [5]:
data.shape

(99221, 9)

# Предобработка

In [7]:
def preprocess(s):
    pattern = '[а-яё]{3,}'
    s = ' '.join(re.findall(pattern, s.lower()))
    return s

Добавим новую колонку с предобработанным текстом

In [8]:
data['item_name'] = data['item_name'].astype('str')
data['item_preproc'] = data['item_name'].apply(preprocess)

In [9]:
data.head()

,receipt_id,receipt_dayofweek,receipt_time,item_name,item_quantity,item_price,item_nds_rate,category_id,brands,item_preproc
0,11,6,20:34,"Молоко 3,2%,шт",2.0,8,2,78,NaN,молоко
1,39,4,11:28,"Компот из изюма, 114 ккал",1.0,4,1,71,NaN,компот изюма ккал
2,39,4,11:28,"Макаронные изделия отварные (масло сливочное),...",1.0,4,1,71,NaN,макаронные изделия отварные масло сливочное ккал
3,56,5,11:42,Кофе Капучино Большой Эден 18,1.0,12,1,70,NaN,кофе капучино большой эден
4,105,3,01:53,Хлеб на СЫВОРОТКЕ 350г,1.0,7,-1,84,NaN,хлеб сыворотке


# Н-граммы

In [10]:
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(data['item_preproc'])

In [11]:
nltk.download('stopwords')
stop = stopwords.words('russian')
stop.append('ккал')
stop.append('шт')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ПКК\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Биграммы

In [12]:
cv = CountVectorizer(ngram_range=(2, 2), stop_words=stop).fit(data['item_preproc'][:25])
print("Размер словаря: ", len(cv.vocabulary_))
print("Словарь:", cv.get_feature_names())

Размер словаря:  44
Словарь: ['авт колор', 'американо средний', 'бедра куриного', 'большой эден', 'бумага туалетная', 'варено копченые', 'вафли топленым', 'вес тортугалия', 'дог куриный', 'изделия отварные', 'капучино большой', 'картофель фри', 'компот изюма', 'копченые мяса', 'кофе американо', 'кофе капучино', 'кубанский молочник', 'кур фил', 'куриного жареное', 'ланчбаскет ориг', 'лоск авт', 'макаронные изделия', 'масло сливочное', 'молоко пастерилиз', 'молоком вес', 'мяса охладенные', 'набер челны', 'нектар мультифрук', 'ориг стрипсы', 'отварные масло', 'пастерилиз рекс', 'рекс бмк', 'сметана кубанский', 'сосиска тесте', 'спинки варено', 'станд картофель', 'стрипсы кур', 'тесте сыром', 'топленым молоком', 'туалетная набер', 'филе бедра', 'хлеб сыворотке', 'хот дог', 'чизбургер луком']


Триграммы

In [13]:
cv = CountVectorizer(ngram_range=(3, 3), stop_words=stop).fit(data['item_preproc'][:25])
print("Размер словаря: ", len(cv.vocabulary_))
print("Словарь:", cv.get_feature_names())

Размер словаря:  26
Словарь: ['бедра куриного жареное', 'бумага туалетная набер', 'варено копченые мяса', 'вафли топленым молоком', 'изделия отварные масло', 'капучино большой эден', 'копченые мяса охладенные', 'кофе американо средний', 'кофе капучино большой', 'ланчбаскет ориг стрипсы', 'лоск авт колор', 'макаронные изделия отварные', 'молоко пастерилиз рекс', 'молоком вес тортугалия', 'ориг стрипсы кур', 'отварные масло сливочное', 'пастерилиз рекс бмк', 'сметана кубанский молочник', 'сосиска тесте сыром', 'спинки варено копченые', 'станд картофель фри', 'стрипсы кур фил', 'топленым молоком вес', 'туалетная набер челны', 'филе бедра куриного', 'хот дог куриный']


# Классификация с Н-граммами

Биграммы

In [14]:
tfidf = TfidfVectorizer(stop_words=stop, max_features=1000, ngram_range=(2, 2))
X = tfidf.fit_transform(data.item_preproc)
y = data.category_id

In [15]:
clf = LogisticRegression(max_iter=100, class_weight='balanced', C = 0.1)
print('Среднее \n',cross_val_score(clf, X, y, cv=3, scoring='f1_weighted'))

[0.39582513 0.35136491 0.33500316]


Триграммы

In [16]:
tfidf = TfidfVectorizer(stop_words=stop, max_features=1000, ngram_range=(3, 3))
X = tfidf.fit_transform(data.item_preproc)

In [17]:
clf = LogisticRegression(max_iter=100, class_weight='balanced', C = 0.1)
print('Среднее \n',cross_val_score(clf, X, y, cv=3, scoring='f1_weighted'))

[0.16931148 0.14017387 0.13285841]


Н-граммы

In [18]:
tfidf = TfidfVectorizer(stop_words=stop, max_features=1000)
X = tfidf.fit_transform(data.item_preproc)

In [19]:
clf = LogisticRegression(max_iter=100, class_weight='balanced', C = 0.1)
print('Среднее \n',cross_val_score(clf, X, y, cv=3, scoring='f1_weighted'))

[0.63725976 0.62463009 0.59416746]


Н-граммы дали лучший результат, по сравнению с би- и триграммами.

# Кодирование категориальных переменных.
Используем категориальную переменную `item_price` для классификации

Dummie

In [71]:
clf = RandomForestClassifier(max_depth = 10)
X = pd.get_dummies(data['item_price'])
print(X.values)

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [72]:
print('Среднее \n', np.mean(cross_val_score(clf, X, y, cv=3, scoring='f1_weighted')))

Среднее 
 0.06353112130333338


OneHotEncoder

In [52]:
ohe = OneHotEncoder().fit(data['item_price'].values.reshape(-1,1))
X = ohe.transform(data['item_price'].values.reshape(-1,1))
print(X[:10])

  (0, 8)	1.0
  (1, 4)	1.0
  (2, 4)	1.0
  (3, 12)	1.0
  (4, 7)	1.0
  (5, 7)	1.0
  (6, 8)	1.0
  (7, 8)	1.0
  (8, 9)	1.0
  (9, 9)	1.0


In [53]:
print('Среднее \n', np.mean(cross_val_score(clf, X, y, cv=3, scoring='f1_weighted')))

Среднее 
 0.06446401821438598


TargetEncoder

In [54]:
import category_encoders as ce
TE = ce.TargetEncoder().fit(data['item_price'].values.reshape(-1,1), y)
X = TE.transform(data['item_price'].values.reshape(-1,1))
print(X[:10])

    0
0   8
1   4
2   4
3  12
4   7
5   7
6   8
7   8
8   9
9   9


In [55]:
print('Среднее \n', np.mean(cross_val_score(clf, X, y, cv=3, scoring='f1_weighted')))

Среднее 
 0.07279764774787857


TargetEncoder справился лучше.

# Используем категориальную переменную `brands` для классификации

Заменим значения NaN на `Без бренда`, чтобы корректно работала модель

In [57]:
data = pd.read_csv('data.csv')
data = data.fillna('Без бренда')
data.head(10)

,receipt_id,receipt_dayofweek,receipt_time,item_name,item_quantity,item_price,item_nds_rate,category_id,brands
0,11,6,20:34,"Молоко 3,2%,шт",2.0,8,2,78,Без бренда
1,39,4,11:28,"Компот из изюма, 114 ккал",1.0,4,1,71,Без бренда
2,39,4,11:28,"Макаронные изделия отварные (масло сливочное),...",1.0,4,1,71,Без бренда
3,56,5,11:42,Кофе Капучино Большой Эден 18,1.0,12,1,70,Без бренда
4,105,3,01:53,Хлеб на СЫВОРОТКЕ 350г,1.0,7,-1,84,Без бренда
5,122,0,11:46,Сосиска в тесте с сыром 1шт ГЕ,3.0,7,2,84,Без бренда
6,129,3,15:17,ЛанчБаскет 5 за 300: 2 шт ОРИГ Стрипсы кур фил,1.0,8,2,69,Без бренда
7,129,3,15:17,Станд Картофель фри,2.0,8,6,69,Без бренда
8,129,3,15:17,Хот-дог Куриный СБ,1.0,9,2,69,Без бренда
9,129,3,15:17,Чизбургер с луком СБ,1.0,9,2,68,Без бренда


In [59]:
data.brands.unique()

array(['Без бренда', 'zewa', 'кока-кола', ..., 'леди джем', 'puff', 'тот'],
      dtype=object)

Dummie 

In [69]:
clf = RandomForestClassifier(max_depth = 10)
X = pd.get_dummies(data['brands'])
print(X.values)

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [70]:
print('Среднее \n', np.mean(cross_val_score(clf, X, y, cv=3, scoring='f1_weighted')))

Среднее 
 0.0640503502198634


OneHotEncoder

In [60]:
ohe = OneHotEncoder().fit(data['brands'].values.reshape(-1,1))
X = ohe.transform(data['brands'].values.reshape(-1,1))
print(X[:10])

  (0, 1376)	1.0
  (1, 1376)	1.0
  (2, 1376)	1.0
  (3, 1376)	1.0
  (4, 1376)	1.0
  (5, 1376)	1.0
  (6, 1376)	1.0
  (7, 1376)	1.0
  (8, 1376)	1.0
  (9, 1376)	1.0


In [61]:
print('Среднее \n', np.mean(cross_val_score(clf, X, y, cv=3, scoring='f1_weighted')))

Среднее 
 0.06381077558093833


TargetEncoder

In [62]:
import category_encoders as ce
TE = ce.TargetEncoder().fit(data['brands'].values.reshape(-1,1), y)
X = TE.transform(data['brands'].values.reshape(-1,1))
print(X[:10])

           0
0  77.532886
1  77.532886
2  77.532886
3  77.532886
4  77.532886
5  77.532886
6  77.532886
7  77.532886
8  77.532886
9  77.532886


In [63]:
print('Среднее \n', np.mean(cross_val_score(clf, X, y, cv=3, scoring='f1_weighted')))

Среднее 
 0.14956715366727383


TargetEncoder справился лучше и даже превзошёл переменную `item_price`.

# Итоги

1. Использование предобработки и Н-грамм помогло достичь точности в 63%, биграммы и триграммы дали меньшую точность.

2. Кодирование и последующая классификация категориальных переменных дали точность хуже, чем при предобработке:

    Использование переменной `item_name` дало результаты в 7% при кодировке TargetEncoder-ом.
    
    Использование переменной `brands` дало результаты в 14%, при кодировке TargetEncoder-ом.

Ансамбли решил не делать, т.к. они вряд ли как-то повлияют на точность.